In [3]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import pandas as pd

In [4]:
ontology = pd.read_csv('lotr/ontology/ontology.csv', sep='\t')
network_3books = pd.read_csv('lotr/tables/networks-id-3books.csv')

In [5]:
ontology.head()
#ontology[ontology['id']=='frod']['Label'].item()
#network_3books.head()


,id,type,Label,FreqSum,subtype,gender
0,andu,pla,Anduin,109,pla,NaN
1,arag,per,Aragorn,1069,men,male
2,arat,per,Arathorn,36,men,male
3,arwe,per,Arwen,51,elves,female
4,bage,pla,Bag End,77,pla,NaN


In [6]:
# split dataset

only_persons = ontology.loc[ontology['type'] == 'per']
only_places = ontology.loc[ontology['type'] == 'pla']
only_other = ontology.loc[ontology['type'] != 'pla']
only_other = only_other.loc[only_other['type'] != 'per']
only_per_pla = pd.concat([only_persons, only_places])
id_col_to_list_per = only_persons['id'].tolist()
id_col_to_list_pla = only_places['id'].tolist()
id_list_per_pla = id_col_to_list_per + id_col_to_list_pla


network_3books_only_per = network_3books
network_3books_only_per = network_3books_only_per.loc[network_3books_only_per['IdSource'].isin(id_col_to_list_per)]
network_3books_only_per = network_3books_only_per.loc[network_3books_only_per['IdTarget'].isin(id_col_to_list_per)]

network_3books_per_pla = network_3books
network_3books_per_pla = network_3books_per_pla.loc[network_3books_per_pla['IdSource'].isin(id_list_per_pla)]
network_3books_per_pla = network_3books_per_pla.loc[network_3books_per_pla['IdTarget'].isin(id_list_per_pla)]


print(len(network_3books))
print(len(network_3books_only_per))
print(len(network_3books_per_pla))
print(len(only_persons))
print(len(only_places))
print(len(only_other))
print(len(only_per_pla))

1444
450
1087
43
24
8
67


In [7]:
only_places.head()
only_places['Label'].unique()
only_other['Label'].unique()
network_3books_per_pla.head()

,IdSource,IdTarget,Weight,Type
0,frod,sams,533,undirected
1,frod,ganda,181,undirected
2,merr,pipp,162,undirected
3,arag,frod,146,undirected
4,frod,goll,127,undirected


In [8]:
#build dataset for barnes hut algorithm github! https://github.com/lsscecilia/GraphVisualisation/tree/main

network_3books_per_pla.head()

with open("per_pla_barnes_hut.txt", "w") as file:

    for index, row in network_3books_per_pla.iterrows():
        string = ""
        string += str(row[0]) + " -- "
        string += str(row[1]) + ","
        string += str(row[2]) + "\n"
        file.write(string)

    



C:\Users\leoni\AppData\Local\Temp\ipykernel_1112\3785055609.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  string += str(row[0]) + " -- "
C:\Users\leoni\AppData\Local\Temp\ipykernel_1112\3785055609.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  string += str(row[1]) + ","
C:\Users\leoni\AppData\Local\Temp\ipykernel_1112\3785055609.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  string += str(row[2]) + "\n"


In [9]:
to_json_pla_per = only_per_pla
to_json_network = network_3books_per_pla

In [10]:
to_json_pla_per.head()

,id,type,Label,FreqSum,subtype,gender
1,arag,per,Aragorn,1069,men,male
2,arat,per,Arathorn,36,men,male
3,arwe,per,Arwen,51,elves,female
5,bali,per,Balin,30,dwarf,male
6,bere,per,Beregond,77,men,male


In [11]:
to_json_network.head()

,IdSource,IdTarget,Weight,Type
0,frod,sams,533,undirected
1,frod,ganda,181,undirected
2,merr,pipp,162,undirected
3,arag,frod,146,undirected
4,frod,goll,127,undirected


In [12]:
#build json file for D3 on oberservable

to_json_pla_per = only_per_pla
to_json_pla_per = to_json_pla_per.rename(columns={"subtype": "group"})
to_json_network = network_3books_per_pla
to_json_network = to_json_network.rename(columns={"IdSource": "source", "IdTarget": "target", "Weight": "value"})



In [13]:
cols_pla_per = ['id', 'group']
cols_network = ['source', 'target', 'value']

to_json_network['json_col'] = to_json_network[cols_network].apply(lambda x: x.to_json(), axis=1)

to_json_pla_per['json_col'] = to_json_pla_per[cols_pla_per].apply(lambda x: x.to_json(), axis=1)
to_json_network.head()
#to_json_pla_per.head()

,source,target,value,Type,json_col
0,frod,sams,533,undirected,"{""source"":""frod"",""target"":""sams"",""value"":533}"
1,frod,ganda,181,undirected,"{""source"":""frod"",""target"":""ganda"",""value"":181}"
2,merr,pipp,162,undirected,"{""source"":""merr"",""target"":""pipp"",""value"":162}"
3,arag,frod,146,undirected,"{""source"":""arag"",""target"":""frod"",""value"":146}"
4,frod,goll,127,undirected,"{""source"":""frod"",""target"":""goll"",""value"":127}"


In [14]:
from json import dump
import json


file = "some.json"
json_file = {}
json_file['nodes'] = []
json_file['links'] = []
for index, row in to_json_pla_per.iterrows():
    json_file['nodes'].append(row[6])

for index, row in to_json_network.iterrows():
    json_file['links'].append(row[4])

r = json.dumps(json_file)
file = r
loaded_r = json.loads(r)
loaded_r['links']

C:\Users\leoni\AppData\Local\Temp\ipykernel_1112\1756954550.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  json_file['nodes'].append(row[6])
C:\Users\leoni\AppData\Local\Temp\ipykernel_1112\1756954550.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  json_file['links'].append(row[4])


['{"source":"frod","target":"sams","value":533}',
 '{"source":"frod","target":"ganda","value":181}',
 '{"source":"merr","target":"pipp","value":162}',
 '{"source":"arag","target":"frod","value":146}',
 '{"source":"frod","target":"goll","value":127}',
 '{"source":"bilb","target":"frod","value":126}',
 '{"source":"ganda","target":"pipp","value":124}',
 '{"source":"arag","target":"ganda","value":123}',
 '{"source":"goll","target":"sams","value":118}',
 '{"source":"frod","target":"pipp","value":115}',
 '{"source":"gimli","target":"lego","value":109}',
 '{"source":"frod","target":"shir","value":104}',
 '{"source":"pipp","target":"sams","value":101}',
 '{"source":"frod","target":"merr","value":86}',
 '{"source":"arag","target":"lego","value":84}',
 '{"source":"arag","target":"sams","value":75}',
 '{"source":"arag","target":"gimli","value":73}',
 '{"source":"ganda","target":"saru","value":72}',
 '{"source":"boro","target":"frod","value":68}',
 '{"source":"arag","target":"merr","value":65}',
 

In [15]:
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%", select_menu=True)

lotr_net.barnes_hut()

Add Nodes first, next code block add edges

In [16]:
# add only persons


def add_node(id, type, label, freqsum, subtype):

    if type == 'per' and len(lotr_net.nodes) < 10:
        lotr_net.add_node(id)
        #lotr_net.add_node(id, size = min(300,freqsum), group=subtype)
        #lotr_net.add_node(id, size = min(400,freqsum), label=label, group=subtype)

        



[add_node(row[0], row[1], row[2], row[3], row[4]) for row in zip(ontology['id'],ontology['type'],ontology['Label'],ontology['FreqSum'],ontology['subtype'])]

lotr_net.nodes
len(lotr_net.nodes)


10

In [17]:
network_3books_only_per.head()

,IdSource,IdTarget,Weight,Type
0,frod,sams,533,undirected
1,frod,ganda,181,undirected
2,merr,pipp,162,undirected
3,arag,frod,146,undirected
4,frod,goll,127,undirected


In [18]:
only_other.tail()

,id,type,Label,FreqSum,subtype,gender
21,ents,gro,Ents,136,ents,NaN
42,hobb,gro,Hobbits,127,hobbit,NaN
54,nazg,gro,Nine Riders,54,men,NaN
59,ring,thin,Ring,427,thing,NaN
74,orcs,gro,Orcs,321,orcs,NaN


In [20]:
#only person edged 
def add_edge(src, dst, w):
    lotr_net.add_edge(src,dst, weight=int(w))

#[add_edge(row[0], row[1], row[2]) for row in zip(network_3books_only_per['IdSource'],network_3books_only_per['IdTarget'],network_3books_only_per['Weight'])]

lotr_net.show_buttons(filter_=["physics"])
lotr_net.show("lotr.html", notebook = False)

lotr.html


First try is following

In [21]:
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%", select_menu=True)

lotr_net.barnes_hut()

def add_edge(src, dst, w):
    lotr_net.add_node(src, label=ontology[ontology['id']==str(src)]['Label'].item())
    lotr_net.add_node(dst)
    #lotr_net.add_node(src, src, title=ontology.query('id==src')['Label'])
    #lotr_net.add_node(dst, dst, title=ontology.query('id==dst')['Label'])
    lotr_net.add_edge(src,dst, weight=int(w))

[add_edge(row[0], row[1], row[2]) for row in zip(network_3books['IdSource'],network_3books['IdTarget'],network_3books['Weight'])]

lotr_net.nodes
lotr_net.show("lotr.html", notebook = False)

lotr.html


In [22]:
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%")

#lotr_net.barnes_hut

for i in range(0,20):
    lotr_net.add_node(network_3books['IdSource'].iloc[i])
    lotr_net.add_node(network_3books['IdTarget'].iloc[i])
    lotr_net.add_edge(network_3books['IdSource'].iloc[i], network_3books['IdTarget'].iloc[i], weight=int(network_3books['Weight'].iloc[i]))




lotr_net.nodes
lotr_net.toggle_physics(True)
lotr_net.show("lotr.html", notebook = False)

lotr.html


## Try to creat graph with Networkx

In [23]:
lotr_graph = nx.Graph()
lotr_graph = nx.Graph()

In [24]:
# pyvis node att: label, title (hover), group, size, color, id
for _, row in only_persons.iterrows():
    if row['id'] == "frod":
        print(row['FreqSum'])
    #lotr_graph.add_node(row['id'], label=row['Label'])
    lotr_graph.add_node(row['id'], label=row['Label'], size=max(5,row['FreqSum']/40), group=row['subtype'], gender=row['gender'])

2040


In [25]:
# create edged by iteration
for _, row in network_3books_only_per.iterrows():
    lotr_graph.add_edge(row['IdSource'], row['IdTarget'], weight=row['Weight']/58, color="gray")


In [26]:
# create edges from dataframe in networkx
lotr_graph = nx.from_pandas_edgelist(network_3books_only_per, "IdSource", "IdTarget", ["Weight"])

In [27]:
lotr_network = Network(height="1000px", width="1000px")
#pos = nx.spring_layout(lotr_graph, iterations=15, seed=1721, weight = "Weight")

#nx.draw(lotr_graph, pos)
#lotr_network = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", select_menu=True, filter_menu=True)
lotr_network.barnes_hut(gravity=-5000, central_gravity=0, spring_length=200, spring_strength=0.009, damping=0.025, overlap=0)
lotr_network.from_nx(lotr_graph)
lotr_network.show_buttons(filter_=['physics'])
lotr_network.toggle_physics(False)

lotr_network.show('lotr_network.html', notebook = False)

lotr_network.html


# Create graph with places

In [28]:
lotr_graph = nx.Graph()
lotr_graph = nx.Graph()

In [29]:
# pyvis node att: label, title (hover), group, size, color, id
for _, row in only_per_pla.iterrows():
    if row['id'] == "frod":
        print(row['FreqSum'])
    #lotr_graph.add_node(row['id'], label=row['Label'])
    lotr_graph.add_node(row['id'], label=row['Label'], size=max(5,row['FreqSum']/40), group=row['subtype'], gender=row['gender'])

2040


In [30]:
# create edged by iteration
for _, row in network_3books_per_pla.iterrows():
    lotr_graph.add_edge(row['IdSource'], row['IdTarget'], weight=row['Weight']/58, color="gray")

In [32]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

In [33]:
# Step 1: Execute SPARQL Query
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?entity ?entityLabel ?entityDescription
WHERE {
  {
    dbr:Albert_Einstein dbo:influenced ?entity .
    ?entity rdfs:label ?entityLabel .
    OPTIONAL { ?entity dbo:abstract ?entityDescription . }
    FILTER (lang(?entityLabel) = 'en')
    FILTER (lang(?entityDescription) = 'en')
  }
  UNION
  {
    ?entity dbo:influencedBy dbr:Albert_Einstein .
    ?entity rdfs:label ?entityLabel .
    OPTIONAL { ?entity dbo:abstract ?entityDescription . }
    FILTER (lang(?entityLabel) = 'en')
    FILTER (lang(?entityDescription) = 'en')
  }
}
LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()


In [34]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
from pyvis.network import Network

# Step 1: Execute SPARQL Query
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?property ?hasValue ?isValueOf
WHERE {
  {
    dbr:Vienna_University_of_Economics_and_Business ?property ?hasValue
  }
  UNION
  {
    ?isValueOf ?property dbr:Vienna_University_of_Economics_and_Business
  }
}
LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Step 2: Process Query Results
G = nx.Graph()
university_node = "Vienna University of Economics and Business"

for result in results["results"]["bindings"]:
    property = result["property"]["value"]
    if "hasValue" in result:
        value = result["hasValue"]["value"]
        G.add_node(value)
        G.add_edge(university_node, value, label=property)
    if "isValueOf" in result:
        value = result["isValueOf"]["value"]
        G.add_node(value)
        G.add_edge(value, university_node, label=property)

# Step 4: Visualize with Pyvis
net = Network(notebook=True, height="750px", width="100%")
net.from_nx(G)
net.show("vienna_university_graph.html", notebook = False)



vienna_university_graph.html


In [39]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
from pyvis.network import Network
from IPython.core.display import display, HTML

# Step 1: Execute SPARQL Query
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?property ?hasValue ?isValueOf
WHERE {
  {
    dbr:Vienna_University_of_Economics_and_Business ?property ?hasValue
  }
  UNION
  {
    ?isValueOf ?property dbr:Vienna_University_of_Economics_and_Business
  }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Step 2: Process Query Results
G = nx.Graph()
university_node = "Vienna University of Economics and Business"

for result in results["results"]["bindings"]:
    property = result["property"]["value"]
    if "hasValue" in result:
        value = result["hasValue"]["value"]
        G.add_node(value)
        G.add_edge(university_node, value, label=property)
    if "isValueOf" in result:
        value = result["isValueOf"]["value"]
        G.add_node(value)
        G.add_edge(value, university_node, label=property)

# Step 4: Visualize with Pyvis in Jupyter Notebook
net = Network(notebook=True, height="750px", width="100%")
net.from_nx(G)
net.show("vienna_university_graph.html", notebook = False)

# Display in Jupyter Notebook
display(HTML("vienna_university_graph.html"))


ImportError: cannot import name 'display' from 'IPython.core.display' (C:\Users\leoni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\IPython\core\display.py)

In [40]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network


In [41]:
ontology = pd.read_csv('lotr/ontology/ontology.csv', sep='\t')
network_3books = pd.read_csv('lotr/tables/networks-id-3books.csv')

In [42]:
ontology.head()
#ontology[ontology['id']=='frod']['Label'].item()
#network_3books.head()


,id,type,Label,FreqSum,subtype,gender
0,andu,pla,Anduin,109,pla,NaN
1,arag,per,Aragorn,1069,men,male
2,arat,per,Arathorn,36,men,male
3,arwe,per,Arwen,51,elves,female
4,bage,pla,Bag End,77,pla,NaN


Add Nodes first, next code block add edges

In [43]:
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%", select_menu=True)

lotr_net.barnes_hut()

In [44]:
# add only persons


def add_node(id, type, label, freqsum, subtype):

    if type == 'per' and len(lotr_net.nodes) < 10:
        lotr_net.add_node(id)
        #lotr_net.add_node(id, size = min(300,freqsum), group=subtype)
        #lotr_net.add_node(id, size = min(400,freqsum), label=label, group=subtype)

        



[add_node(row[0], row[1], row[2], row[3], row[4]) for row in zip(ontology['id'],ontology['type'],ontology['Label'],ontology['FreqSum'],ontology['subtype'])]

lotr_net.nodes
len(lotr_net.nodes)


10

In [45]:
# split dataset

only_persons = ontology.loc[ontology['type'] == 'per']
only_placed = ontology.loc[ontology['type'] == 'pla']
only_other = ontology.loc[ontology['type'] != 'pla']
only_other = only_other.loc[only_other['type'] != 'per']
id_col_to_list = only_persons['id'].tolist()


list_of_node_val = lotr_net.nodes
network_3books_only_per = network_3books
network_3books_only_per = network_3books_only_per.loc[network_3books_only_per['IdSource'].isin(id_col_to_list)]
network_3books_only_per = network_3books_only_per.loc[network_3books_only_per['IdTarget'].isin(id_col_to_list)]

print(len(network_3books))
print(len(network_3books_only_per))
print(len(only_persons))
print(len(only_placed))
print(len(only_other))

1444
450
43
24
8


In [46]:
network_3books_only_per.head()

,IdSource,IdTarget,Weight,Type
0,frod,sams,533,undirected
1,frod,ganda,181,undirected
2,merr,pipp,162,undirected
3,arag,frod,146,undirected
4,frod,goll,127,undirected


In [47]:
only_other.tail()

,id,type,Label,FreqSum,subtype,gender
21,ents,gro,Ents,136,ents,NaN
42,hobb,gro,Hobbits,127,hobbit,NaN
54,nazg,gro,Nine Riders,54,men,NaN
59,ring,thin,Ring,427,thing,NaN
74,orcs,gro,Orcs,321,orcs,NaN


In [51]:
# only person edges
label_lookup = ontology.set_index('id')['Label']

def ensure_node(node_id):
    if lotr_net.get_node(node_id) is None:
        label = label_lookup.get(node_id, node_id)
        lotr_net.add_node(node_id, label=label)

def add_edge(src, dst, w):
    ensure_node(src)
    ensure_node(dst)
    lotr_net.add_edge(src, dst, weight=int(w))

#[add_edge(row[0], row[1], row[2]) for row in zip(network_3books_only_per['IdSource'], network_3books_only_per['IdTarget'], network_3books_only_per['Weight'])]

lotr_net.show_buttons(filter_=["physics"])
lotr_net.show("lotr.html", notebook=False)

lotr.html


First try is following

In [52]:
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%", select_menu=True)

lotr_net.barnes_hut()

def add_edge(src, dst, w):
    lotr_net.add_node(src, label=ontology[ontology['id']==str(src)]['Label'].item())
    lotr_net.add_node(dst)
    #lotr_net.add_node(src, src, title=ontology.query('id==src')['Label'])
    #lotr_net.add_node(dst, dst, title=ontology.query('id==dst')['Label'])
    lotr_net.add_edge(src,dst, weight=int(w))

[add_edge(row[0], row[1], row[2]) for row in zip(network_3books['IdSource'],network_3books['IdTarget'],network_3books['Weight'])]

lotr_net.nodes
lotr_net.show("lotr.html", notebook = False)

lotr.html


In [ ]:
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%")

#lotr_net.barnes_hut

for i in range(0,20):
    lotr_net.add_node(network_3books['IdSource'].iloc[i])
    lotr_net.add_node(network_3books['IdTarget'].iloc[i])
    lotr_net.add_edge(network_3books['IdSource'].iloc[i], network_3books['IdTarget'].iloc[i], weight=int(network_3books['Weight'].iloc[i]))




lotr_net.nodes
lotr_net.toggle_physics(True)
lotr_net.show("lotr.html", notebook = False)

lotr.html


## Try to creat graph with Networkx

In [53]:
lotr_graph = nx.Graph()
lotr_graph = nx.Graph()

In [54]:
# pyvis node att: label, title (hover), group, size, color, id
for _, row in only_persons.iterrows():
    if row['id'] == "frod":
        print(row['FreqSum'])
    #lotr_graph.add_node(row['id'], label=row['Label'])
    lotr_graph.add_node(row['id'], label=row['Label'], size=max(5,row['FreqSum']/40), group=row['subtype'], gender=row['gender'])

2040


In [55]:
# create edged by iteration
for _, row in network_3books_only_per.iterrows():
    lotr_graph.add_edge(row['IdSource'], row['IdTarget'], weight=row['Weight']/58, color="gray")


In [56]:
# create edges from dataframe in networkx
lotr_graph = nx.from_pandas_edgelist(network_3books_only_per, "IdSource", "IdTarget", ["Weight"])

In [57]:
lotr_network = Network(height="1000px", width="1000px")
#pos = nx.spring_layout(lotr_graph, iterations=15, seed=1721, weight = "Weight")

#nx.draw(lotr_graph, pos)
#lotr_network = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", select_menu=True, filter_menu=True)
lotr_network.barnes_hut(gravity=-5000, central_gravity=0, spring_length=200, spring_strength=0.009, damping=0.025, overlap=0)
lotr_network.from_nx(lotr_graph)
lotr_network.show_buttons(filter_=['physics'])
lotr_network.toggle_physics(True)

lotr_network.show('lotr_network.html', notebook = False)
lotr_network.barnes_hut(gravity=-5000, central_gravity=0, spring_length=400, spring_strength=0.009, damping=0.025, overlap=0)
lotr_network.from_nx(lotr_graph)
lotr_network.show_buttons(filter_=['physics'])
lotr_network.toggle_physics(False)

lotr_network.show('lotr_network.html', notebook = False)

lotr_network.html


TypeError: 'barnesHut' object is not callable

In [58]:
only_persons['gender'].unique()
only_persons['subtype'].unique()

array(['men', 'elves', 'dwarf', 'hobbit', 'animal', 'ainur', 'orcs',
       'ents'], dtype=object)

In [59]:
race_colors = {'men':'#7A84DD','dwarf':"#B15B60", 'elves':'#8ACAE5', 'hobbit':'#BD9267', 'ents':'#F1A54D', 
               'orcs': "#020104",'ainur':'#3A7575',"animal":"#8ACAE5","ents":"#E3845D"}

In [ ]:
#race_colors = {'men':'#7A84DD','unknown':'#7A84DD', 'dwarves':"#B15B60", 'elves':'#8ACAE5', 'hobbits':'#BD9267', 'ents':'#F1A54D', 
               'orcs': "#020104",'ainur':'#3A7575',"half-elven":"#8ACAE5","ents":"#E3845D","dragons":"#F10301"
               ,"trolls":"#863E34"}

In [60]:
import re

def add_physics_stop_to_html(filepath):
    with open(filepath, 'r', encoding="utf-8") as file:
        content = file.read()

    # Search for the stabilizationIterationsDone event and insert the network.setOptions line
    pattern = r'(network.once\("stabilizationIterationsDone", function\(\) {)'
    replacement = r'\1\n\t\t\t\t\t\t  // Disable the physics after stabilization is done.\n\t\t\t\t\t\t  network.setOptions({ physics: false });'

    new_content = re.sub(pattern, replacement, content, flags=re.DOTALL)

    # Write the modified content back to the file
    with open(filepath, 'w', encoding="utf-8") as file:
        file.write(new_content)


In [62]:
from tkinter import W
from pyvis.network import Network
from collections import OrderedDict
from unidecode import unidecode
import urllib

lotr_graph = nx.Graph()
lotr_graph = nx.Graph()

G = Network(height="800px", width="100%", bgcolor="#222222", font_color="white",notebook=True)

for _, row in only_persons.iterrows():
    weight = row['FreqSum']
    if weight > 15:
        character = row['id']
        race = row['subtype']
        if race in race_colors.keys():
            color = race_colors[race]
        else:
            color = '#7A84DD'
    else: 
        color = None
    
    G.add_node(character, label=row['Label'], weight = weight, size=max(10,row['FreqSum']/40), gender=row['gender'], color = color)
        


for index, row in network_3books_only_per.iterrows():
    weight = row['Weight']
    if weight > 1:
        character_0 = row['IdSource']
        character_1 = row['IdTarget']
        G.add_edge(character_0,
                    character_1,
                    weight=weight,
                    color="gray")
        #width = 1.5
        

neighbor_map = G.get_adj_list()

'''
for node in G.nodes:
    temp_df = characters_details_df[characters_details_df['normlized_name'] ==node["id"]].drop(['character_name',
                                                                                   'culture',
                                                                                   'weapon',
                                                                                   "normalized_realm",
                                                                                   "normlized_name"],axis=1)
    temp_df['graph_neighbors'] = ', '.join([str(elem) for elem in neighbor_map[node["id"]]]) 
    temp_df_name = temp_df['lotr_name']
    temp_df.drop(labels=['lotr_name'], axis=1,inplace = True)
    temp_df.insert(0, 'lotr_name', temp_df_name)
    temp_df = temp_df.T.dropna().rename({'normalized_culture': 'culture','normalized_weapon': "weapon","lotr_name":"name"})
    temp_df = temp_df.rename({0:''},axis=1)
    
    node["title"] += temp_df.to_html(header=False)
    node["value"] = len(neighbor_map[node["id"]])
'''
G.barnes_hut(gravity=-5000, central_gravity=0, spring_length=200, spring_strength=0.009, damping=0.025, overlap=0)
G.write_html('lotr_network_graph.html')
#G.toggle_physics(False)
add_physics_stop_to_html("lotr_network_graph.html")
#G.show('lotr_network_graph.html', notebook = False)







In [63]:
# write-html.py
import webbrowser
import os
print ()


#Change path to reflect file location
filename = 'file:///'+os.getcwd()+'/' + 'lotr_network_graph.html'
webbrowser.open_new_tab(filename)

True

In [64]:
G.show_buttons(filter_=['physics'])
G.show('lotr_network_graph.html', notebook = False)

lotr_network_graph.html


In [ ]:
import gravis as pv

ASAHHH Everythign copied, have to split stuff now

In [66]:
from SPARQLWrapper import SPARQLWrapper, JSON
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network
import pandas as pd
ontology = pd.read_csv('lotr/ontology/ontology.csv', sep='\t')
network_3books = pd.read_csv('lotr/tables/networks-id-3books.csv')
ontology.head()
#ontology[ontology['id']=='frod']['Label'].item()
#network_3books.head()

# split dataset

only_persons = ontology.loc[ontology['type'] == 'per']
only_places = ontology.loc[ontology['type'] == 'pla']
only_other = ontology.loc[ontology['type'] != 'pla']
only_other = only_other.loc[only_other['type'] != 'per']
id_col_to_list_per = only_persons['id'].tolist()
id_col_to_list_pla = only_places['id'].tolist()
id_list_per_pla = id_col_to_list_per + id_col_to_list_pla


list_of_node_val = lotr_net.nodes
network_3books_only_per = network_3books
network_3books_only_per = network_3books_only_per.loc[network_3books_only_per['IdSource'].isin(id_col_to_list_per)]
network_3books_only_per = network_3books_only_per.loc[network_3books_only_per['IdTarget'].isin(id_col_to_list_per)]

network_3books_per_pla = network_3books
network_3books_per_pla = network_3books_per_pla.loc[network_3books_per_pla['IdSource'].isin(id_list_per_pla)]
network_3books_per_pla = network_3books_per_pla.loc[network_3books_per_pla['IdTarget'].isin(id_list_per_pla)]


print(len(network_3books))
print(len(network_3books_only_per))
print(len(network_3books_per_pla))
print(len(only_persons))
print(len(only_places))
print(len(only_other))
only_places.head()
only_places['Label'].unique()
#Add Nodes first, next code block add edges
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%", select_menu=True)

lotr_net.barnes_hut()
# add only persons


def add_node(id, type, label, freqsum, subtype):

    if type == 'per' and len(lotr_net.nodes) < 10:
        lotr_net.add_node(id)
        #lotr_net.add_node(id, size = min(300,freqsum), group=subtype)
        #lotr_net.add_node(id, size = min(400,freqsum), label=label, group=subtype)

        



[add_node(row[0], row[1], row[2], row[3], row[4]) for row in zip(ontology['id'],ontology['type'],ontology['Label'],ontology['FreqSum'],ontology['subtype'])]

lotr_net.nodes
len(lotr_net.nodes)

network_3books_only_per.head()
only_other.tail()
#only person edged 
def add_edge(src, dst, w):
    lotr_net.add_edge(src,dst, weight=int(w))

#[add_edge(row[0], row[1], row[2]) for row in zip(network_3books_only_per['IdSource'],network_3books_only_per['IdTarget'],network_3books_only_per['Weight'])]

lotr_net.show_buttons(filter_=["physics"])
lotr_net.show("lotr.html", notebook = False)
#First try is following
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%", select_menu=True)

lotr_net.barnes_hut()

def add_edge(src, dst, w):
    lotr_net.add_node(src, label=ontology[ontology['id']==str(src)]['Label'].item())
    lotr_net.add_node(dst)
    #lotr_net.add_node(src, src, title=ontology.query('id==src')['Label'])
    #lotr_net.add_node(dst, dst, title=ontology.query('id==dst')['Label'])
    lotr_net.add_edge(src,dst, weight=int(w))

[add_edge(row[0], row[1], row[2]) for row in zip(network_3books['IdSource'],network_3books['IdTarget'],network_3books['Weight'])]

lotr_net.nodes
lotr_net.show("lotr.html", notebook = False)
#adapt example of Game of Thrones on Pyvis page
lotr_net = Network(width="100%")

#lotr_net.barnes_hut

for i in range(0,20):
    lotr_net.add_node(network_3books['IdSource'].iloc[i])
    lotr_net.add_node(network_3books['IdTarget'].iloc[i])
    lotr_net.add_edge(network_3books['IdSource'].iloc[i], network_3books['IdTarget'].iloc[i], weight=int(network_3books['Weight'].iloc[i]))




lotr_net.nodes
lotr_net.toggle_physics(True)
lotr_net.show("lotr.html", notebook = False)
## Try to creat graph with Networkx
lotr_graph = nx.Graph()
lotr_graph = nx.Graph()
# pyvis node att: label, title (hover), group, size, color, id
for _, row in only_persons.iterrows():
    if row['id'] == "frod":
        print(row['FreqSum'])
    #lotr_graph.add_node(row['id'], label=row['Label'])
    lotr_graph.add_node(row['id'], label=row['Label'], size=max(5,row['FreqSum']/40), group=row['subtype'], gender=row['gender'])
# create edged by iteration
for _, row in network_3books_only_per.iterrows():
    lotr_graph.add_edge(row['IdSource'], row['IdTarget'], weight=row['Weight']/58, color="gray")

# create edges from dataframe in networkx
lotr_graph = nx.from_pandas_edgelist(network_3books_only_per, "IdSource", "IdTarget", ["Weight"])
lotr_network = Network(height="1000px", width="1000px")
#pos = nx.spring_layout(lotr_graph, iterations=15, seed=1721, weight = "Weight")

#nx.draw(lotr_graph, pos)
#lotr_network = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", select_menu=True, filter_menu=True)
lotr_network.barnes_hut(gravity=-5000, central_gravity=0, spring_length=200, spring_strength=0.009, damping=0.025, overlap=0)
lotr_network.from_nx(lotr_graph)
lotr_network.show_buttons(filter_=['physics'])
lotr_network.toggle_physics(False)

lotr_network.show('lotr_network.html', notebook = False)


1444
450
1087
43
24
8
lotr.html
lotr.html
lotr.html
2040
lotr_network.html
